# SyftBox Installer Tutorial v2

This notebook demonstrates the new `syft-installer` that follows the official `install.sh` implementation.

## Key Features

- 🎯 **Follows install.sh exactly** - Uses the same flow as the official installer
- 🖥️ **Terminal mode** - Delegates to `syftbox login` TUI when in terminal
- 📓 **Notebook support** - Falls back to programmatic auth in Jupyter/Colab
- 🌍 **Environment variables** - Supports all install.sh environment variables

## Installation

First, let's install the library:

In [ ]:
# Install from local directory (for testing)
!pip install -e /Users/atrask/Documents/GitHub/syft-installer

# Or install from PyPI when published
# !pip install syft-installer

## 1. Quick Installation (Follows install.sh)

The simplest way to install SyftBox. In notebooks, this will use programmatic authentication:

In [ ]:
import syft_installer as si

# This follows install.sh flow:
# - Downloads binary
# - Sets up environment
# - In terminal: runs 'syftbox login' TUI
# - In notebook: uses programmatic auth
si.install()

Label(value='test')

Output()

## 2. Installation Modes (from install.sh)

The installer supports the same modes as install.sh:

In [ ]:
import syft_installer as si

# Download binary only (no setup)
si.install(install_mode="download-only")
print("✅ Binary downloaded to ~/.local/bin/syftbox")

In [ ]:
# Setup only (assumes binary already exists)
si.install(install_mode="setup-only")
print("✅ Setup completed")

In [ ]:
# Interactive mode (default) - full installation
si.install(install_mode="interactive")

## 3. Environment Variables (Matching install.sh)

The installer respects all environment variables from install.sh:

In [ ]:
import os
import syft_installer as si

# Set environment variables (just like with install.sh)
os.environ["INSTALL_MODE"] = "interactive"
os.environ["INSTALL_APPS"] = "dataroom,whisper"
os.environ["DEBUG"] = "true"

# Install will use these environment variables
si.install()

## 4. App Installation

Install SyftBox apps during setup (like install.sh with INSTALL_APPS):

In [ ]:
import syft_installer as si

# Install with specific apps
si.install(install_apps="dataroom,whisper")

# Or use environment variable
# export INSTALL_APPS=dataroom,whisper

## 5. Debug Mode

Enable debug output to see what's happening (like install.sh with DEBUG=true):

In [ ]:
import syft_installer as si

# Enable debug mode
si.install(debug=True)

# Or use environment variable
# export DEBUG=true

## 6. Pre-configured Installation

For notebook environments, you can pre-configure email to skip that prompt:

In [1]:
import syft_installer as si

# Pre-configure email (notebook environments only)
installer = si.Installer(email="your.email@example.com")
installer.install()  # Will only prompt for OTP

Label(value='test')

Output()

HTML(value='<div style="color: red;">✗ Installation failed: Failed to request OTP: Expecting value: line 1 col…

AuthenticationError: Failed to request OTP: Expecting value: line 1 column 1 (char 0)

## 7. Programmatic Authentication (Notebooks Only)

In notebook environments where the TUI cannot run, you can use programmatic auth:

In [ ]:
import syft_installer as si

# Create installer
installer = si.Installer()

# Download and setup
installer.download_binary()
installer.setup_environment()

# Programmatic authentication
email = "your.email@example.com"
installer.request_otp(email)
print(f"✅ OTP sent to {email}")

# Get OTP from user
otp = input("Enter OTP from email: ")
installer.verify_otp(otp)
print("✅ Authentication successful")

# Start client
installer.start_client(background=True)
print("✅ Client started")

## 8. Check Installation Status

In [ ]:
import syft_installer as si

# Check if installed
if si.is_installed():
    print("✅ SyftBox is installed")
    
    # Load configuration
    config = si.load_config()
    if config:
        print(f"📧 Logged in as: {config.email}")
        print(f"📁 Data directory: {config.data_dir}")
        print(f"🌐 Server: {config.server_url}")
        print(f"🔧 Binary at: {config.binary_path}")
else:
    print("❌ SyftBox is not installed")

# Check if running
if si.is_running():
    print("✅ SyftBox client is running")
else:
    print("❌ SyftBox client is not running")

## 9. Managing the Client

In [ ]:
import syft_installer as si

# Start client (uses existing config)
si.start_client(background=True)
print("✅ Client started")

# Check status
if si.is_running():
    print("✅ Client is running")

# Stop client
si.stop_client()
print("✅ Client stopped")

## 10. Headless/CI Mode

For automated environments:

In [ ]:
import syft_installer as si

def automated_install(email: str, otp_getter):
    """Example of headless installation."""
    installer = si.Installer(
        email=email,
        headless=True,
        otp_callback=otp_getter
    )
    
    # This will:
    # 1. Download binary
    # 2. Setup environment
    # 3. Request OTP
    # 4. Call otp_getter() to get the OTP
    # 5. Verify and save config
    installer.install()
    
    # Start in background
    installer.start_client(background=True)
    
    return True

# Example usage:
# def get_otp_from_email():
#     # Your code to fetch OTP from email
#     return "ABCD1234"
# 
# automated_install("user@example.com", get_otp_from_email)

## 11. Platform and Environment Info

In [ ]:
from syft_installer.platform import get_platform_info, get_binary_url
from syft_installer.runtime import RuntimeEnvironment

# Platform info
os_name, arch = get_platform_info()
print(f"🖥️  OS: {os_name}")
print(f"🏗️  Architecture: {arch}")
print(f"📥 Binary URL: {get_binary_url()}")

# Runtime environment
runtime = RuntimeEnvironment()
print(f"\n📓 Running in notebook: {runtime.is_notebook}")
print(f"🌐 Running in Colab: {runtime.is_colab}")
print(f"💻 Has terminal: {runtime.has_tty}")
print(f"🎛️  Can use widgets: {runtime.can_use_widgets}")
print(f"📁 Default data dir: {runtime.default_data_dir}")

## Key Differences from v1

1. **Follows install.sh**: The installer now follows the exact flow of the official installer
2. **Delegates to TUI**: In terminals, it runs `syftbox login` instead of reimplementing auth
3. **Installation modes**: Supports download-only, setup-only, and interactive modes
4. **Environment variables**: Respects INSTALL_MODE, INSTALL_APPS, DEBUG, etc.
5. **PATH management**: Automatically updates shell profiles
6. **App installation**: Can install apps during setup
7. **Old version removal**: Removes old installations via uv/pip

## Troubleshooting

### PATH not updated?
The installer adds `~/.local/bin` to your PATH in shell profiles. You may need to:
```bash
source ~/.bashrc  # or ~/.zshrc
```

### Binary not found?
Make sure the download completed:
```python
si.install(install_mode="download-only")
```

### Debug mode
Enable debug output:
```python
si.install(debug=True)
```